In [12]:
# destytojo pvz

import PySimpleGUI as sg

layout = [
    [sg.Text("Koks tavo vardas?", font="Verdana 15")],
    [sg.Input(key="-NAME-", font="Terminal 15")],
    [
        sg.Button("Pasisveikinti", key="-HELLO-"), 
        sg.Button("Atsisveikinti", key="-BYE-"),
    ],
    [sg.Text(size=(40, 1), key="-OUTPUT-", font="Verdana 15")],
]

window = sg.Window("LABAS", layout)
sg.DEFAULT_FONT

while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED:
        break

    if event == "-HELLO-":
        window["-OUTPUT-"].update(
            f"Sveiki {values['-NAME-']}",
            text_color="#99ff99"
        )
    elif event == "-BYE-":
        window["-OUTPUT-"].update(
            f"Viso gero {values['-NAME-']}",
            text_color="#ff9999"
        )

window.close()

In [4]:
# mano - veikia labai primityvus!

import PySimpleGUI as sg

layout = [
    [sg.Text("Koks jusu vardas, ar galite paskolinti pinigu?")],
    [sg.Input(key="-VARDAS-")],
    [
        sg.Button("Negaliu!", key="-ATE-"),
        sg.Button("Galiu!", key="-VALIO-")
    ]
]

layout1 = [
    [sg.Text("Kiek?")],
    [sg.Input(key="-SUMA-")],
    [
        sg.Button("Skolinu!", key="-EXIT-"),
    ]
]

window = sg.Window("Labas drauge", layout)
window1 = sg.Window("Skolinama suma €", layout1)

while True:
    event, values = window.read()
    if event == "-ATE-":
        break
    event, values = window1.read()
    if event == "-EXIT-":
        break



window.close()
window1.close()

# Destytojo zaidimas X ir O

In [2]:
import PySimpleGUI as sg

def get_player(turn):
    if turn % 2 == 0:
        return "X"
    return "O"

def check_winning(game):
    winning_combos = [
        ['1x1', '1x2', '1x3'], ['2x1', '2x2', '2x3'], ['3x1', '3x2', '3x3'],
        ['1x1', '2x1', '3x1'], ['1x2', '2x2', '3x2'], ['1x3', '2x3', '3x3'],
        ['1x1', '2x2', '3x3'], ['1x3', '2x2', '3x1']
    ]
    for combo in winning_combos:
        if combo[0] in game and combo[1] in game and combo[2] in game \
            and game[combo[0]] == game[combo[1]] and game[combo[0]] == game[combo[2]]:
            return True
    return False

def reset_game(window: sg.Window):
    window['-CURRENT-'].update("Current player: X")
    for row in range(1, 4):
        for col in range(1, 4):
            window[f"{row}x{col}"].update(" ", disabled=False)
    window['-RESET-'].update(visible=False)
    return 0, {}

coordinates = []
rows = []
for row in range(1, 4):
    columns = []
    for col in range(1, 4):
        columns.append(sg.Button("   ", key=f"{row}x{col}", font="Terminal 20"))
        coordinates.append(f"{row}x{col}")
    rows.append(columns)

layout = [
    [sg.Text("Current player: X", key="-CURRENT-")],
    rows,
    [sg.Button('Play Again', key='-RESET-', visible=False)]
]

window = sg.Window("X's and O's", layout)

turn = 0
game = {}

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break

    if event in coordinates:
        window[event].update(get_player(turn), disabled=True)
        game[event] = get_player(turn)
        is_game_won = check_winning(game)
        if is_game_won or turn >= 8:
            if is_game_won:
                window['-CURRENT-'].update(f"WINNER: {get_player(turn)}")
            else:
                window['-CURRENT-'].update('DRAW')
            window['-RESET-'].update(visible=True)
        else:
            turn += 1
            window['-CURRENT-'].update(f"Current player: {get_player(turn)}")

    if event == '-RESET-':
        turn, game = reset_game(window)

## LOAN klase

In [6]:
from math import ceil

class Loan():
    def __init__(
            self, 
            loan: float, 
            interest_percent: float = 4, 
            duration: int = 12,
            downpayment: float = 0, 
        ) -> None:
        self.loan = loan
        self.downpayment = downpayment
        self.interest_percent = interest_percent
        self.duration = duration
        self.principal = loan - downpayment
        self.monthly_true = self.get_monthly_true()
        self.monthly = self.get_monthly_lowest()
        self.repayable = round(self.monthly_true * self.duration, 2)
        self.loan_total = downpayment + self.repayable
        self.increase = self.repayable - self.principal
        self.remaining = self.repayable
        self.penny_months = self.get_penny_months()
        self.payments = self.get_payment_chart()

    def get_monthly_true(self) -> float:
        monthly_interest = self.interest_percent / 12 / 100
        monthly = self.principal * monthly_interest \
            / (1 - (1 + monthly_interest) ** -self.duration)
        return monthly
    
    def get_monthly_lowest(self) -> float:
        return int(self.monthly_true * 100) / 100
    
    def get_penny_months(self) -> int:
        if self.repayable == self.monthly * self.duration:
            return 0
        else:
            pennies = self.repayable - self.monthly * self.duration
            return int(pennies * 100)
        
    def get_monthly(self, month=1):
        if month <= self.penny_months:
            return self.monthly + 0.01
        else:
            return self.monthly

    def get_payment_chart(self) -> list:
        payments = []
        for month in range(self.duration):
            payments.append(self.get_monthly(month+1))
        return payments

    def print_loan_info(self) -> None:
        print(f"""------ Loan stats -----
            Loan:           {self.loan:10.2f}
            Downpayment:    {self.downpayment:10.2f}
            Interest:     {self.interest_percent:10.2f} %
            Duration:        {self.duration} months
            Principal:      {self.principal:10.2f}
            Monthly:        {self.monthly:10.2f}
            Pennies:        {self.get_penny_months():10d}
            Loan Total:     {self.loan_total:10.2f}
            Increase:       {self.increase:10.2f}\n""")

    def print_monthly_payments(self) -> None:
        for period, payment in enumerate(self.payments):
            print(f"{period+1:3d}: {payment:10.2f}")

# LOAN calculator

In [7]:
import PySimpleGUI as sg
# from loan import Loan # nereikia nes LOAN klase nurodyta virsuje
from typing import Any

def main_window() -> None:
    layout = [
        [sg.Text("Loan amount", size=20), sg.Input(key="-LOAN-", justification="right", size=15)],
        [sg.Text("Duration (months)", size=20), sg.Input("12", key="-DURATION-", justification="right", size=15)],
        [sg.Text("Interest, %", size=20), sg.Input("5.0", key="-INTEREST-", justification="right", size=15)],
        [sg.Text("Downpayment", size=20), sg.Input("0", key="-DOWNPAYMENT-", justification="right", size=15)],
        [sg.Button("Calculate", key="-CALCULATE-", pad=8), sg.Button("Show Chart", key="-SHOW-CHART-", pad=8, disabled=True)],
        [sg.Text("", key="-ERRORS-", size=35, justification="center", text_color='#FF9999')],
        [sg.Text("Monthly Payment", size=20), sg.Text("", key="-MONTHLY-", justification="right", size=15)],
        [sg.Text("Price Increase", size=20), sg.Text("", key="-INCREASE-", justification="right", size=15)],
        [sg.Text("Total Reoayment", size=20), sg.Text("", key="-REPAYABLE-", justification="right", size=15)],
        [sg.Text("Total loan", size=20), sg.Text("", key="-TOTAL-", justification="right", size=15)],
    ]
    window = sg.Window("Loan Calculator", layout)
    loan = None
    while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED:
            break
        if event == '-CALCULATE-':
            loan = calculate_loan(values, window)
        if event == '-SHOW-CHART-':
            if isinstance(loan, Loan):
                chart_window(loan, window)

def calculate_loan(values: Any, window: sg.Window) -> Loan:
    window['-ERRORS-'].update("")
    try:
        loan = Loan(
            float(values['-LOAN-']), 
            float(values['-INTEREST-']), 
            int(values['-DURATION-']), 
            float(values['-DOWNPAYMENT-']),
        )
    except Exception as error:
        window['-ERRORS-'].update("Values must be numbers")
    else:
        window['-MONTHLY-'].update(f"{loan.monthly:,.2f}")
        window['-INCREASE-'].update(f"{loan.increase:,.2f}")
        window['-REPAYABLE-'].update(f"{loan.repayable:,.2f}")
        window['-TOTAL-'].update(f"{loan.loan_total:,.2f}")
        window['-SHOW-CHART-'].update(disabled=False)
        return loan

def chart_window(loan: Loan, main_window: sg.Window) -> None:
    main_window.hide()
    payments = []
    for month, payment in enumerate(loan.get_payment_chart()):
        payments.append([month+1, f"{payment:.2f}"])
    layout = [
        [sg.Table(
            values=payments, 
            headings=['Month', 'Amount'], 
            key='-CHART-TABLE-', 
            col_widths=[5, 15], 
            auto_size_columns=False
        )],
        [sg.Button('Close', key='-CLOSE-')],
    ]
    window = sg.Window('Payment Chart', layout)
    while True:
        event, _ = window.read()
        if event in (sg.WIN_CLOSED, '-CLOSE-'):
            break
    window.close()
    main_window.un_hide()

main_window()

In [13]:
# mano - upgrade - nebaigtas !

import PySimpleGUI as sg

layout = [
    [sg.Text("Koks jusu vardas?"), sg.Input(key="-VARDAS-", justification="centre")],
    [sg.Text("Ar galite paskolinti pinigu? Pasirinkite zemiau,aciu!")],
    [sg.Button("Negaliu!", key="-ATE-", pad=20), sg.Button("Galiu!", key="-VALIO-")],
    [sg.Text("Kokia suma?"), sg.Input(key="-SUMA-", justification="right")],
    [sg.Button("Skolinu!", key="-EXIT")]
    
]

window = sg.Window("Labas drauge", layout)

while True:
    event, values = window.read()
    if event == "-ATE-":
        break
    if event == "-VALIO-":
        continue
    else:
        event == "-EXIT-"
        break


window.close()